# 02.2 Introduction to ETL with PySpark & Minio

In this laboratory we will be learning the basics about using PySpark to implement ETL pipelines with a booking database as an example

![schema](imgs/bookings-schema.png)

## 1. Moving data from local storage to Minio

In this task you will need to move the files located in `data/bookings` of the local storage to the Minio bucket named `test`, all the files should have the prefix `lab2/landing/` using Apache Spark

In [ ]:
s3_landing_prefix = "lab2/landing"
local_file_path = "data/bookings"

In [ ]:
# Create and configure your spark session here

In [ ]:
files = ["bookings", "facilities", "members"]

for file in files:
    # upload the file to Minio
    pass

## 2. Transforming the data

In this section you will have to read the files you moved from the local storage to a landing zone in the `test` bucket and apply some transformations to them. 

Then you will save the transformed data 

PySpark provides multiple functions to trasnform the data, a majority of these are provided in the `pyspark.sql.functions` module

### Column & DataFrame Manipulation

| Function   | Purpose                                 | Example                                                        |
| ---------- | --------------------------------------- | -------------------------------------------------------------- |
| `col`      | Reference a column by name              | `df.select(col("age"))`                                        |
| `lit`      | Create a column with a literal value    | `df.withColumn("country", lit("USA"))`                         |
| `alias`    | Rename a column in a select             | `df.select(col("age").alias("user_age"))`                      |
| `when`     | Conditional expressions (like SQL CASE) | `df.select(when(col("age") > 18, "Adult").otherwise("Minor"))` |
| `coalesce` | Return first non-null value             | `df.select(coalesce(col("phone"), lit("N/A")))`                |


### Aggregation & Grouping

| Function        | Purpose                  | Example                                 |
| --------------- | ------------------------ | --------------------------------------- |
| `count`         | Count rows               | `df.groupBy("country").agg(count("*"))` |
| `countDistinct` | Count distinct values    | `df.agg(countDistinct("user_id"))`      |
| `sum`           | Sum of a column          | `df.agg(sum("sales"))`                  |
| `avg` / `mean`  | Average of a column      | `df.agg(avg("salary"))`                 |
| `max` / `min`   | Maximum or minimum value | `df.agg(max("salary"), min("salary"))`  |


### String Functions
| Function         | Purpose                      | Example                                                            |
| ---------------- | ---------------------------- | ------------------------------------------------------------------ |
| `lower`          | Convert to lowercase         | `df.select(lower(col("name")))`                                    |
| `upper`          | Convert to uppercase         | `df.select(upper(col("name")))`                                    |
| `concat`         | Concatenate columns          | `df.select(concat(col("first_name"), lit(" "), col("last_name")))` |
| `concat_ws`      | Concatenate with a separator | `df.select(concat_ws("-", "year", "month", "day"))`                |
| `substring`      | Extract substring            | `df.select(substring(col("phone"), 1, 3))`                         |
| `trim`           | Trim whitespace              | `df.select(trim(col("username")))`                                 |
| `regexp_extract` | Extract regex match          | `df.select(regexp_extract(col("email"), r"@(.+)", 1))`             |
| `regexp_replace` | Replace regex match          | `df.select(regexp_replace(col("phone"), "-", ""))`                 |


### Date & Time Functions

| Function                      | Purpose                                | Example                                                               |
| ----------------------------- | -------------------------------------- | --------------------------------------------------------------------- |
| `current_date`                | Current date                           | `df.select(current_date())`                                           |
| `current_timestamp`           | Current timestamp                      | `df.select(current_timestamp())`                                      |
| `date_add`                    | Add days to date                       | `df.select(date_add(col("start_date"), 7))`                           |
| `date_sub`                    | Subtract days from date                | `df.select(date_sub(col("start_date"), 7))`                           |
| `datediff`                    | Difference between two dates (in days) | `df.select(datediff(col("end_date"), col("start_date")))`             |
| `months_between`              | Difference in months                   | `df.select(months_between(col("end_date"), col("start_date")))`       |
| `year`, `month`, `dayofmonth` | Extract date parts                     | `df.select(year(col("date")), month(col("date")))`                    |
| `to_date`                     | Convert string to date                 | `df.select(to_date(col("date_string"), "yyyy-MM-dd"))`                |
| `to_timestamp`                | Convert string to timestamp            | `df.select(to_timestamp(col("datetime_str"), "yyyy-MM-dd HH:mm:ss"))` |


### Null handling
| Function                           | Purpose                    | Example                               |
| ---------------------------------- | -------------------------- | ------------------------------------- |
| `isnull`                           | Check for null values      | `df.filter(col("email").isNull())`    |
| `isnotnull`                        | Check for non-null values  | `df.filter(col("email").isNotNull())` |
| `na.fill` *(method, not function)* | Replace nulls with a value | `df.na.fill({"email": "unknown"})`    |
| `na.drop` *(method, not function)* | Drop rows with nulls       | `df.na.drop()`                        |


In [ ]:
import pyspark.sql.functions as F

transformed_prefix = "lab2/transformed"

### 2.1 Transforming members data

For members we need to transform the original table so:
- Shows the full name of the members
- Remove the `surname` and `firstname` columns
- The `recommendedby` column should show the full name of the person instead of the id, if the person was not recommended show "NOT RECOMMENDED"


In [ ]:
# Create a spark dataframe from the members file uploaded in s3

In [ ]:
# Transform the spark dataframe

In [ ]:
# You can see the results with df.show()

In [ ]:
# Save the dataframe to Minio

### 2.2 Transforming the bookings

We need a new bookings table that shows

- `bookid` : booking id
- `memberid`: member id
- `member` : the full name of the member that made the booking
- `facilityid`: facility id
- `facility`: the name of the facility
- `starttime`: start time of the booking
- `endtime`: end time of the booking
- `slots`: amount of hours of the booking
- `cost`: the cost of the booking, the per hour cost by the total of hours


In [ ]:
# Load all the dataframes you need from Minio

In [ ]:
# Create the new dataframe

In [ ]:
# Save the result dataframe

## 3. Reporting 

You will have to generate reports with the following information:

- 3.1 `facility_report_monthly`: For each facility the total bookings, hours booked and revenuee (per month) ordered by facility
- 3.2 `facility_report_yearly`: For each facility the total bookings, hours booked and revenue
- 3.3 `member_report_monthly`: For each member the total boookings, total hours used, total money spent, and members recommended (per month)
- 3.4 `member_report_yearly`: For each member the total hours used and members recommended

Only consider the 2012 year

You can use any data that you want, save the reports with the prefix `lab2/reports`


In [ ]:
s3_reports_prefix = "lab2/reports"

Using the bookings report we built in the previous section we already have some of the work done

In [ ]:
# Load your transformed bookings dataframe here!

## 3.1 Facility monthly report

For breaking down the report into months we will need a rolling dates table, this a table where we will have all the possible dates in a given period of time. In this case we will cosider the entire 2012 year.

In [ ]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

# Generate all the possible dates (remember we only need a month) here ...

# dates_df = ...

## 3.2 Facility yearly report

This is quite simple, just an aggregation over the monthly report

## 3.3 Members monthly report

Like the facility one but with a little twist

## 3.4 Members yearly report

You should be able to do this very easily